In [1]:
!pip install pyspark


In [2]:
import socket, time, threading

def start_socket_server():
    host = "localhost"
    port = 9999
    s = socket.socket()
    s.bind((host, port))
    s.listen(1)
    conn, addr = s.accept()

    messages = [
        "spark streaming dstream",
        "spark spark streaming",
        "big data spark"
    ]

    while True:
        for msg in messages:
            conn.send((msg + "\n").encode())
            time.sleep(2)

threading.Thread(target=start_socket_server, daemon=True).start()


In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext.getOrCreate()
ssc = StreamingContext(sc, 5)


/usr/local/lib/python3.12/dist-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [4]:
lines = ssc.socketTextStream("localhost", 9999)


In [5]:
words = lines.flatMap(lambda line: line.split(" "))
pairs = words.map(lambda w: (w, 1))
counts = pairs.reduceByKey(lambda a, b: a + b)


In [6]:
counts.pprint()

ssc.start()
ssc.awaitTerminationOrTimeout(30)
ssc.stop(stopSparkContext=False)


-------------------------------------------
Time: 2026-01-11 17:05:50
-------------------------------------------
('streaming', 1)
('dstream', 1)
('spark', 1)

-------------------------------------------
Time: 2026-01-11 17:05:55
-------------------------------------------
('streaming', 2)
('big', 1)
('dstream', 1)
('spark', 4)
('data', 1)

-------------------------------------------
Time: 2026-01-11 17:06:00
-------------------------------------------
('streaming', 1)
('big', 1)
('spark', 3)
('data', 1)

-------------------------------------------
Time: 2026-01-11 17:06:05
-------------------------------------------
('streaming', 2)
('dstream', 1)
('big', 1)
('spark', 4)
('data', 1)

-------------------------------------------
Time: 2026-01-11 17:06:10
-------------------------------------------
('streaming', 2)
('dstream', 1)
('spark', 3)

-------------------------------------------
Time: 2026-01-11 17:06:15
-------------------------------------------
('big', 1)
('streaming', 2)
('ds

In [9]:
if 'ssc' in globals() and ssc._jssc is not None:
    try:
        # Attempt to stop the StreamingContext. If it's already stopped or in a bad state,
        # ssc.stop() might throw an exception, which we catch.
        ssc.stop(stopSparkContext=False)
    except Exception as e:
        # Handle cases where ssc might be in a bad state or already stopped but not cleanly
        print(f"Warning: Could not gracefully stop ssc. Attempting to create new one. Error: {e}")

# Re-initialize StreamingContext using the existing SparkContext (sc was kept alive by stopSparkContext=False)
from pyspark.streaming import StreamingContext
ssc = StreamingContext(sc, 5)

lines = ssc.textFileStream("/content/input")

/usr/local/lib/python3.12/dist-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [10]:
lines = ssc.textFileStream("/content/input")


In [11]:
ssc = StreamingContext(sc, 5)


/usr/local/lib/python3.12/dist-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(
